In [1]:
import pickle
import os
from itertools import product
import tensorflow as tf
import math

In [5]:
import tensorflow as tf
from image_utils import load_grayscale_image
import numpy as np
from skimage import filters

In [131]:
model = tf.keras.models.load_model('training_results_2\\evaluated_model_27022024_182439.h5')

In [132]:
model.input

[<KerasTensor: shape=(None, 150, 220, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 531) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_3')>]

In [162]:
bachnorm = model.layers[2]

In [168]:
model.layers[1].output

<KerasTensor: shape=(None, 75, 110, 12) dtype=float32 (created by layer 'conv2d')>

In [133]:
first_conv_layer = model.layers[1]

In [134]:
first_conv_layer.output_shape

(None, 75, 110, 12)

In [257]:
intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[1].output)

In [258]:
intermediate_layer_model

In [287]:
some_image = load_grayscale_image('preprocessed_images_protocol_2\\data_protocol_2_preprocessing\\protocol_2_preprocessed_images\\2\\cf-2-2.jpg')

In [288]:
some_image.shape

(150, 220)

In [290]:
np.min(some_image)

0

In [261]:
normalized_image = some_image / 255.0

In [262]:
input = np.expand_dims(normalized_image, axis=-1)

In [263]:
input1 = np.ones((531,))
input2 = np.ones((1,))

In [264]:
input_max = np.expand_dims(np.ones_like(normalized_image), axis=-1)
input_min = np.expand_dims(np.zeros_like(normalized_image), axis=-1)

In [265]:
max_intermediate_output = intermediate_layer_model([tf.expand_dims(input_max, 0), input1, input2])
min_intermediate_output = intermediate_layer_model([tf.expand_dims(input_min, 0), input1, input2])

In [266]:
intermediate_output = intermediate_layer_model([tf.expand_dims(input, 0), input1, input2])

In [267]:
numpis = intermediate_output.numpy()
numpis_max = max_intermediate_output.numpy()
numpis_min = min_intermediate_output.numpy()

In [269]:
numpis_reshaped = numpis.squeeze()
numpis_reshaped_max = numpis_max.squeeze()
numpis_reshaped_min = numpis_min.squeeze()

In [275]:
np.min(numpis_reshaped_max)

0.0

In [277]:
np.max(numpis_reshaped_min)

0.06850497

In [271]:
numpis_reshaped_max.shape

(75, 110, 12)

In [146]:
numpis_reshaped_transposed = np.transpose(numpis_reshaped, (2, 0, 1))

In [147]:
numpis_reshaped_transposed.shape

(12, 75, 110)

In [124]:
image = some_image / 255.0
h, w = image.shape
Ix = np.gradient(image, axis=1)
Iy = np.gradient(image, axis=0)
Ixx = np.gradient(Ix, axis=1)
Ixy = np.gradient(Ix, axis=0)
Iyy = np.gradient(Iy, axis=0)
gradient_magnitude = np.sqrt(Ix**2 + Iy**2)
gradient_direction = np.arctan2(Iy, Ix)
xn = np.tile(np.arange(w) / w, (h, 1))
yn = np.tile(np.arange(h) / h, (w, 1)).T

In [125]:
feature_map = np.stack([
        image, Ix, Iy, Ixx, Ixy, Iyy, gradient_magnitude, gradient_direction, xn, yn
    ], axis=0)

In [126]:
feature_map.shape

(10, 150, 220)

In [127]:
feature_map.reshape(10, -1).shape

(10, 33000)

In [128]:
feature_map_reshaped = np.transpose(feature_map, (1, 2, 0))

In [129]:
feature_map.shape

(10, 150, 220)

In [130]:
feature_map_reshaped.shape

(150, 220, 10)

In [32]:
path_cedar_gen = 'C:\\Users\\ester\\faculdade\\tcc\\database_cedar\\signatures\\full_org'
path_cedar_forg = 'C:\\Users\\ester\\faculdade\\tcc\\database_cedar\\signatures\\full_forg'

In [36]:
lista = os.listdir(path_cedar_gen)[:]

In [37]:
lista.pop()

'Thumbs.db'

In [38]:
len(lista)

1320

In [30]:
input_layer = tf.keras.layers.Input(shape=(210, 308, 1))

conv1 = tf.keras.layers.Conv2D(96, (11, 11), activation='relu', strides=4, padding='same')(input_layer)
batch1 = tf.keras.layers.BatchNormalization()(conv1)

pool1 = tf.keras.layers.MaxPooling2D((3, 3), strides=2)(batch1)

conv2 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(pool1)
batch3 = tf.keras.layers.BatchNormalization()(conv2)

pool2 = tf.keras.layers.MaxPooling2D((3, 3), strides=2)(batch3)

conv3 = tf.keras.layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(pool2)
batch5 = tf.keras.layers.BatchNormalization()(conv3)

conv4 = tf.keras.layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(batch5)
batch6 = tf.keras.layers.BatchNormalization()(conv4)

conv5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(batch6)
#conv5 = tf.keras.layers.Conv2D(300, (3, 3), activation='relu', strides=1, padding='same')(batch6)
batch7 = tf.keras.layers.BatchNormalization()(conv5)

pool3 = tf.keras.layers.MaxPooling2D((3, 3), strides=2, padding='same')(batch7)

flatten = tf.keras.layers.Flatten()(pool3)
#flatten = tf.keras.layers.Flatten()(batch7)

l2 = tf.keras.regularizers.L2(l2=1e-4)

dense1 = tf.keras.layers.Dense(3234, activation='relu')(flatten)
batch9 = tf.keras.layers.BatchNormalization()(dense1)
dense2 = tf.keras.layers.Dense(3234, activation='relu')(batch9)
batch10 = tf.keras.layers.BatchNormalization()(dense2)

output1 = tf.keras.layers.Dense(531, activation='softmax', kernel_regularizer=l2, name='output1')(batch10)

In [20]:
model = tf.keras.Model(inputs=[input_layer], outputs=[output1])

In [27]:
conv5.shape

TensorShape([None, 12, 18, 256])

In [3]:
dictionaries_of_covariance_matrices_path = 'dictionaries-of-covariance-matrices\\'

In [7]:
m_p_values = [(1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (3, 3)]
max_iterations = [3000, 5000, 7000, 9000]

combinations = list(product(m_p_values, max_iterations))

len(combinations)

32

In [4]:
count = 0
data = []

for subdir, _, files in os.walk(dictionaries_of_covariance_matrices_path):

    for file in files:
        with open(os.path.join(dictionaries_of_covariance_matrices_path, file), 'rb') as pickle_file:
            loaded_data = pickle.load(pickle_file)
            data.extend(loaded_data)

        count += 1
        if count == 3:
            break

In [18]:
dict = data[0]

In [27]:
type(dict['forged_scm'][0])

numpy.ndarray

In [1]:
import numpy as np
from skimage.transform import resize

In [ ]:
def process_images(images):
    processed_images = []
    for image in images:
        height, width = image.shape[:2]
        if height > 800 or width > 800:
            max_dim = max(height, width)
            pad_height = max(0, max_dim - height)
            pad_width = max(0, max_dim - width)
            padded_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant', constant_values=0)
            resized_image = resize(padded_image, (800, 800), anti_aliasing=True)
        else:
            pad_height = max(0, 800 - height)
            pad_width = max(0, 800 - width)
            resized_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant', constant_values=0)
        processed_images.append(resized_image)
    return processed_images

In [7]:
import random

def divide_into_parts(lst, num_parts):
    random.shuffle(lst)  # Shuffle the list randomly
    part_size = len(lst) // num_parts
    remainder = len(lst) % num_parts
    parts = [lst[i * part_size + min(i, remainder):(i + 1) * part_size + min(i + 1, remainder)] for i in range(num_parts)]
    return parts

# Example usage:
# lst is your list of integers
lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
num_parts = 5
parts = divide_into_parts(lst, num_parts)
print(parts)


[[8, 9], [4, 2], [10, 5], [1, 6], [3, 7]]


In [2]:
with open(os.path.join('scm_new_features_gpds_prot_2_supernova', 'scm_gpds_protocol_2_new_features_1-400.pkl'), 'rb') as pickle_file:
    loaded_data = pickle.load(pickle_file)

In [3]:
loaded_data.keys()

dict_keys([82, 356, 350, 151, 28, 59, 15, 41, 75, 128, 42, 17, 395, 200, 307, 46, 79, 81, 68, 127, 369, 111, 281, 216, 37, 261, 243, 279, 396, 120, 236, 230, 211, 124, 101, 131, 93, 378, 203, 118, 229, 241, 125, 357, 237, 210, 117, 22, 218, 53, 342, 179, 391, 95, 126, 239, 163, 108, 193, 33, 271, 313, 197, 94, 389, 362, 290, 267, 60, 262, 245, 6, 368, 160, 372, 132, 381, 99, 228, 87, 170, 173, 168, 220, 259, 123, 254, 385, 66, 274, 250, 48, 272, 109, 153, 167, 54, 11, 215, 1, 370, 165, 149, 231, 122, 251, 181, 277, 67, 249, 217, 133, 224, 191, 143, 152, 195, 192, 58, 175, 10, 80, 104, 335, 343, 306, 61, 263, 246, 182, 183, 252, 29, 146, 40, 196, 4, 71, 107, 145, 161, 375, 202, 140, 296, 139, 226, 26, 240, 162, 43, 322, 233, 21, 105, 100, 45, 30, 371, 174, 78, 399, 278, 227, 180, 366, 315, 91, 98, 171, 235, 189, 347, 89, 256, 39, 363, 134, 150, 367, 136, 141, 69, 340, 284, 8, 238, 355, 204, 155, 329, 299, 352, 344, 358, 223, 242, 330, 320, 190, 52, 309, 74, 77, 294, 64, 266, 72, 351, 17

In [6]:
all_matrices = [gen for gen in loaded_data[10]['forged_scm']]
all_mat_np = np.array(all_matrices)
print(all_mat_np.shape)
np.max(all_mat_np)

(30, 48, 48)


2213.1208

In [185]:
np.max(loaded_data[24]['genuine_scm'][0])

0.004908392

In [29]:
loaded_data['fold_results'][2]['test_users']

[1361,
 1138,
 2638,
 2648,
 1432,
 1095,
 1362,
 1447,
 1475,
 1494,
 2457,
 1167,
 1207,
 1480,
 1311,
 1085,
 1398,
 2488,
 2522,
 1457,
 1173,
 1276,
 1081,
 1287,
 1477,
 1451,
 1415]

In [44]:
import numpy as np

In [48]:
loaded_data[1].keys()

dict_keys(['individual', 'genuine_signatures', 'forged_signatures', 'genuine_scm', 'forged_scm'])

In [5]:
39 % 20

19

In [33]:
loaded_data['fold_results'][2]['aucs']

[0.5436375, 0.57975, 0.568325, 0.5372750000000001, 0.5497749999999999]

In [6]:
for data in loaded_data:
    print(data['individual'], end=', ')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 102, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 103, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 104, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 105, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 106, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 107, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 108, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 109, 1090, 

In [14]:
dico = {'oi': 'hi'}

In [ ]:
listo = [1, 2, 3]

In [15]:
seto = {'oi', 'ola'}

In [ ]:
seto

In [ ]:
import numpy as np
from scipy import ndimage
from PIL import Image

def calculate_center_of_mass(image):
    # Calculate center of mass
    center_of_mass = ndimage.measurements.center_of_mass(image)
    return center_of_mass

def shift_image(image, shift):
    # Shift the image
    shifted_image = ndimage.shift(image, shift)
    return shifted_image

def centerize_image(image_path, output_size):
    # Read the image
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img_array = np.array(img)

    # Calculate center of mass
    center_of_mass = calculate_center_of_mass(img_array)

    # Calculate shift needed
    shift = np.array(output_size) / 2 - np.array(center_of_mass)

    # Shift the image
    shifted_image = shift_image(img_array, shift)

    # Create a larger canvas
    larger_canvas = np.zeros(output_size, dtype=np.uint8)

    # Calculate position to paste the shifted image onto the larger canvas
    paste_position = tuple((np.array(output_size) - np.array(img_array.shape)) // 2)

    # Paste the shifted image onto the larger canvas
    larger_canvas[paste_position[0]:paste_position[0]+img_array.shape[0], 
                  paste_position[1]:paste_position[1]+img_array.shape[1]] = shifted_image

    # Convert back to image
    centered_image = Image.fromarray(larger_canvas)

    return centered_image

# Example usage:
image_path = 'your_image.jpg'  # Path to your input image
output_size = (800, 600)  # Desired output size of the canvas
centered_image = centerize_image(image_path, output_size)
centered_image.show()  # Display the resulting image
